In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

temp_df=pd.read_csv("ExtractedText.csv",encoding='cp1252')
temp_df.shape

(478, 6)

In [ ]:
from datasets import load_dataset
issues_dataset = load_dataset("csv", data_files="ExtractedText.csv",encoding='cp1252',split="train")
issues_dataset

Dataset({
    features: ['DocumentID', 'PassageID', 'Title', 'Subheading', 'Paragraph', 'URL'],
    num_rows: 478
})

In [ ]:
columns = issues_dataset.column_names
columns_to_keep = ['DocumentID', 'PassageID', 'Title', 'Subheading', 'Paragraph','URL']
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
issues_dataset = issues_dataset.remove_columns(columns_to_remove)
issues_dataset

Dataset({
    features: ['DocumentID', 'PassageID', 'Title', 'Subheading', 'Paragraph', 'URL'],
    num_rows: 478
})

In [ ]:
issues_dataset.set_format("pandas")
df = issues_dataset[:]
df

,DocumentID,PassageID,Title,Subheading,Paragraph,URL
0,0,1,Anxiety Disorders (for Parents) - Nemours Kid...,What Are Anxiety Disorders?,Anxiety disorders cause extreme fear and worry...,https://kidshealth.org/en/parents/anxiety-diso...
1,0,2,Anxiety Disorders (for Parents) - Nemours Kid...,What Are the Kinds of Anxiety Disorders?,Different anxiety disorders can affect kids an...,https://kidshealth.org/en/parents/anxiety-diso...
2,0,3,Anxiety Disorders (for Parents) - Nemours Kid...,What Are the Signs & Symptoms of Anxiety?,A parent or teacher may see signs that a child...,https://kidshealth.org/en/parents/anxiety-diso...
3,0,4,Anxiety Disorders (for Parents) - Nemours Kid...,What Causes Anxiety Disorders?,Several things play a role in causing the over...,https://kidshealth.org/en/parents/anxiety-diso...
4,0,5,Anxiety Disorders (for Parents) - Nemours Kid...,How Are Anxiety Disorders Diagnosed?,Anxiety disorders can be diagnosed by a traine...,https://kidshealth.org/en/parents/anxiety-diso...
...,...,...,...,...,...,...
473,88,13,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Toddler Milestones,20. “We blow bubbles to calm [them] down and r...,https://www.mother.ly/toddler/toddler-mileston...
474,88,14,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Viral & Trending,21. “I’ve learned it’s different for each chil...,https://www.mother.ly/toddler/toddler-mileston...
475,88,15,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Toddler Milestones,Related Stories,https://www.mother.ly/toddler/toddler-mileston...
476,88,16,23 Mom-Tested Ways to Manage a Tantrum - Motherly,We believe in creating a world in which all mo...,Contact Us P: Advertise with Us P: Share Your ...,https://www.mother.ly/toddler/toddler-mileston...


In [ ]:
#Rows that had issue, we delete them
l=[0,37,215,249,257,258]
df = df.drop(labels=l, axis=0)
df

,DocumentID,PassageID,Title,Subheading,Paragraph,URL
1,0,2,Anxiety Disorders (for Parents) - Nemours Kid...,What Are the Kinds of Anxiety Disorders?,Different anxiety disorders can affect kids an...,https://kidshealth.org/en/parents/anxiety-diso...
2,0,3,Anxiety Disorders (for Parents) - Nemours Kid...,What Are the Signs & Symptoms of Anxiety?,A parent or teacher may see signs that a child...,https://kidshealth.org/en/parents/anxiety-diso...
3,0,4,Anxiety Disorders (for Parents) - Nemours Kid...,What Causes Anxiety Disorders?,Several things play a role in causing the over...,https://kidshealth.org/en/parents/anxiety-diso...
4,0,5,Anxiety Disorders (for Parents) - Nemours Kid...,How Are Anxiety Disorders Diagnosed?,Anxiety disorders can be diagnosed by a traine...,https://kidshealth.org/en/parents/anxiety-diso...
5,0,6,Anxiety Disorders (for Parents) - Nemours Kid...,How Are Anxiety Disorders Treated?,"Most often, anxiety disorders are treated with...",https://kidshealth.org/en/parents/anxiety-diso...
...,...,...,...,...,...,...
473,88,13,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Toddler Milestones,20. “We blow bubbles to calm [them] down and r...,https://www.mother.ly/toddler/toddler-mileston...
474,88,14,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Viral & Trending,21. “I’ve learned it’s different for each chil...,https://www.mother.ly/toddler/toddler-mileston...
475,88,15,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Toddler Milestones,Related Stories,https://www.mother.ly/toddler/toddler-mileston...
476,88,16,23 Mom-Tested Ways to Manage a Tantrum - Motherly,We believe in creating a world in which all mo...,Contact Us P: Advertise with Us P: Share Your ...,https://www.mother.ly/toddler/toddler-mileston...


In [ ]:
df.shape

(472, 6)

In [ ]:
from datasets import Dataset

comments_dataset = Dataset.from_pandas(df)
comments_dataset

Dataset({
    features: ['DocumentID', 'PassageID', 'Title', 'Subheading', 'Paragraph', 'URL', '__index_level_0__'],
    num_rows: 472
})

In [ ]:
def concatenate_text(examples):
    return {
        "text": examples["Title"]
        + " \n "
        + examples["Subheading"]
        + " \n "
        + examples["Paragraph"]
    }


comments_dataset = comments_dataset.map(concatenate_text)

Map:   0%|          | 0/472 [00:00<?, ? examples/s]

In [ ]:
comments_dataset

Dataset({
    features: ['DocumentID', 'PassageID', 'Title', 'Subheading', 'Paragraph', 'URL', '__index_level_0__', 'text'],
    num_rows: 472
})

In [ ]:
columns = comments_dataset.column_names
columns_to_keep = ["DocumentID","Title","URL"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
document_Dataset = comments_dataset.remove_columns(columns_to_remove)
document_Dataset

Dataset({
    features: ['DocumentID', 'Title', 'URL'],
    num_rows: 472
})

In [ ]:
columns = comments_dataset.column_names
columns_to_keep = ["Title", "Subheading", "Paragraph","URL","text"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
comments_dataset = comments_dataset.remove_columns(columns_to_remove)
comments_dataset

Dataset({
    features: ['Title', 'Subheading', 'Paragraph', 'URL', 'text'],
    num_rows: 472
})

Here we are cleaning, that is removing P: and spaces in the rows.

In [ ]:
comments_dataset.set_format("pandas")
temp_df = comments_dataset[:]

In [ ]:
index_removal=[]
for i in range(472):
  temp_df['text'][i] = temp_df['text'][i].strip().replace("P:","")

for i in range(472):
  temp_df['Paragraph'][i] = temp_df['Paragraph'][i].strip().replace("P:","")
  if len(temp_df['Paragraph'][i])<10:
    index_removal.append(i)
  

In [ ]:
print(index_removal)

[184, 231, 253, 294, 302, 311, 319, 327, 343, 344, 345, 346, 354, 375, 399]


In [ ]:
temp_df=temp_df.drop(index_removal)

In [ ]:
temp_df

,Title,Subheading,Paragraph,URL,text
0,Anxiety Disorders (for Parents) - Nemours Kid...,What Are the Kinds of Anxiety Disorders?,Different anxiety disorders can affect kids an...,https://kidshealth.org/en/parents/anxiety-diso...,Anxiety Disorders (for Parents) - Nemours Kid...
1,Anxiety Disorders (for Parents) - Nemours Kid...,What Are the Signs & Symptoms of Anxiety?,A parent or teacher may see signs that a child...,https://kidshealth.org/en/parents/anxiety-diso...,Anxiety Disorders (for Parents) - Nemours Kid...
2,Anxiety Disorders (for Parents) - Nemours Kid...,What Causes Anxiety Disorders?,Several things play a role in causing the over...,https://kidshealth.org/en/parents/anxiety-diso...,Anxiety Disorders (for Parents) - Nemours Kid...
3,Anxiety Disorders (for Parents) - Nemours Kid...,How Are Anxiety Disorders Diagnosed?,Anxiety disorders can be diagnosed by a traine...,https://kidshealth.org/en/parents/anxiety-diso...,Anxiety Disorders (for Parents) - Nemours Kid...
4,Anxiety Disorders (for Parents) - Nemours Kid...,How Are Anxiety Disorders Treated?,"Most often, anxiety disorders are treated with...",https://kidshealth.org/en/parents/anxiety-diso...,Anxiety Disorders (for Parents) - Nemours Kid...
...,...,...,...,...,...
467,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Toddler Milestones,20. “We blow bubbles to calm [them] down and r...,https://www.mother.ly/toddler/toddler-mileston...,23 Mom-Tested Ways to Manage a Tantrum - Mothe...
468,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Viral & Trending,21. “I’ve learned it’s different for each chil...,https://www.mother.ly/toddler/toddler-mileston...,23 Mom-Tested Ways to Manage a Tantrum - Mothe...
469,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Toddler Milestones,Related Stories,https://www.mother.ly/toddler/toddler-mileston...,23 Mom-Tested Ways to Manage a Tantrum - Mothe...
470,23 Mom-Tested Ways to Manage a Tantrum - Motherly,We believe in creating a world in which all mo...,Contact Us Advertise with Us Share Your Stor...,https://www.mother.ly/toddler/toddler-mileston...,23 Mom-Tested Ways to Manage a Tantrum - Mothe...


In [ ]:
temp_df.to_csv("CleanedText.csv")

In [ ]:
comments_dataset = Dataset.from_pandas(temp_df)
comments_dataset

Dataset({
    features: ['Title', 'Subheading', 'Paragraph', 'URL', 'text', '__index_level_0__'],
    num_rows: 457
})

In [ ]:
columns = comments_dataset.column_names
columns_to_keep = ["Title", "Subheading", "Paragraph","URL","text"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
comments_dataset = comments_dataset.remove_columns(columns_to_remove)
comments_dataset

Dataset({
    features: ['Title', 'Subheading', 'Paragraph', 'URL', 'text'],
    num_rows: 457
})

In [ ]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)


In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [ ]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    #encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
embedding = get_embeddings(comments_dataset["text"][0])
embedding.shape

torch.Size([1, 768])

In [ ]:
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

Map:   0%|          | 0/457 [00:00<?, ? examples/s]

In [ ]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['Title', 'Subheading', 'Paragraph', 'URL', 'text', 'embeddings'],
    num_rows: 457
})

In [ ]:
embeddings_dataset

Dataset({
    features: ['Title', 'Subheading', 'Paragraph', 'URL', 'text', 'embeddings'],
    num_rows: 457
})

In [ ]:
# loading library
import pickle
# create an iterator object with write permission - model.pkl
with open('MultiQA_PassageLevel_pkl', 'wb') as files:
    pickle.dump(embeddings_dataset, files)

# We will save/send this file to the Data Base.

Creating Document embeddings

In [ ]:
import pandas as pd
doc_Df = pd.DataFrame(columns=['DocID','URL','Embeddings'])

In [ ]:
doc_Df['DocID'] = df['DocumentID'].unique()
doc_Df['DocID']

0      0
1      1
2      2
3      3
4      4
      ..
64    84
65    85
66    86
67    87
68    88
Name: DocID, Length: 69, dtype: int64

In [188]:
temp=df.loc[df['DocumentID'] == 1]
temp['DocumentID'].sum()

7

In [ ]:
embeddings_dataset.set_format("pandas")
Document_Temp = embeddings_dataset[:]
Document_Temp

,Title,Subheading,Paragraph,URL,text,embeddings
0,Anxiety Disorders (for Parents) - Nemours Kid...,What Are the Kinds of Anxiety Disorders?,Different anxiety disorders can affect kids an...,https://kidshealth.org/en/parents/anxiety-diso...,Anxiety Disorders (for Parents) - Nemours Kid...,"[-0.01890669, -0.49167317, -0.20652804, -0.001..."
1,Anxiety Disorders (for Parents) - Nemours Kid...,What Are the Signs & Symptoms of Anxiety?,A parent or teacher may see signs that a child...,https://kidshealth.org/en/parents/anxiety-diso...,Anxiety Disorders (for Parents) - Nemours Kid...,"[0.047935653, -0.60621387, -0.22229323, 0.1613..."
2,Anxiety Disorders (for Parents) - Nemours Kid...,What Causes Anxiety Disorders?,Several things play a role in causing the over...,https://kidshealth.org/en/parents/anxiety-diso...,Anxiety Disorders (for Parents) - Nemours Kid...,"[-0.12154601, -0.6636507, -0.19422671, 0.13888..."
3,Anxiety Disorders (for Parents) - Nemours Kid...,How Are Anxiety Disorders Diagnosed?,Anxiety disorders can be diagnosed by a traine...,https://kidshealth.org/en/parents/anxiety-diso...,Anxiety Disorders (for Parents) - Nemours Kid...,"[0.004511428, -0.4369652, -0.2055371, 0.265108..."
4,Anxiety Disorders (for Parents) - Nemours Kid...,How Are Anxiety Disorders Treated?,"Most often, anxiety disorders are treated with...",https://kidshealth.org/en/parents/anxiety-diso...,Anxiety Disorders (for Parents) - Nemours Kid...,"[-0.03623768, -0.5331508, -0.23160432, -0.0326..."
...,...,...,...,...,...,...
452,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Toddler Milestones,20. “We blow bubbles to calm [them] down and r...,https://www.mother.ly/toddler/toddler-mileston...,23 Mom-Tested Ways to Manage a Tantrum - Mothe...,"[0.27846962, -0.4221728, -0.22933762, -0.32362..."
453,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Viral & Trending,21. “I’ve learned it’s different for each chil...,https://www.mother.ly/toddler/toddler-mileston...,23 Mom-Tested Ways to Manage a Tantrum - Mothe...,"[0.36313942, -0.3771683, -0.23246408, -0.29345..."
454,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Toddler Milestones,Related Stories,https://www.mother.ly/toddler/toddler-mileston...,23 Mom-Tested Ways to Manage a Tantrum - Mothe...,"[0.30182707, -0.18127394, -0.28959137, -0.2463..."
455,23 Mom-Tested Ways to Manage a Tantrum - Motherly,We believe in creating a world in which all mo...,Contact Us Advertise with Us Share Your Stor...,https://www.mother.ly/toddler/toddler-mileston...,23 Mom-Tested Ways to Manage a Tantrum - Mothe...,"[0.33036467, -0.25193867, -0.25336376, -0.3638..."


In [ ]:
while i < len(Document_temp.index)-1:
  #present and next same, we add and then average
  


457
